

---


# <h1 align=center> **Carolina del Valle Garay** </h1>




---


# <h1 align=center> **Modelo de Machine Learning** </h1>




---



Se trabajará con los siguientes métodos y técnicas de Procesamiento de Lenguaje Natural:

- TF-IDF (Term Frequency-Inverse Document Frequency): Este método convierte el texto en una representación numérica (matriz de características), donde cada término en el texto se asocia con un valor que refleja su importancia en el documento en comparación con el corpus. Los valores calculados son puramente textuales, basados en la frecuencia de palabras.

- Similitud del Coseno: Esta técnica mide la similitud entre dos vectores de TF-IDF, cuantificando cuán similares son en el espacio vectorial. No se ve afectada por valores numéricos externos, sino únicamente por los términos del texto y sus respectivas ponderaciones.

##Ingesta de datos luego de aplicado el EDA

In [1]:
#Importar librerías
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!gdown https://drive.google.com/file/d/17GdfOWQhKz3vH2wBDVdw-k_1_TBh6o8U/view?usp=sharing --fuzzy

Downloading...
From: https://drive.google.com/uc?id=17GdfOWQhKz3vH2wBDVdw-k_1_TBh6o8U
To: /content/movies_final.parquet
100% 20.6M/20.6M [00:00<00:00, 37.8MB/s]


In [3]:
df = pd.read_parquet('/content/movies_final.parquet')

In [ ]:
df.dtypes

In [ ]:
df['title'][:30]

In [ ]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'name_coll', 'name_gen',
       'production_companies', 'production_countries', 'languages', 'return',
       'release_year', 'actors', 'director'],
      dtype='object')

In [ ]:
#revisar valores nulos
df.isnull().sum()

budget                      0
id                          0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24958
title                       0
vote_average                0
vote_count                  0
name_coll               40860
name_gen                    0
production_companies    11788
production_countries     6207
languages                3765
return                      0
release_year                0
actors                      0
director                  835
dtype: int64

In [ ]:
df.shape

(45345, 22)

**Se crearán dos columnas como resultado del análisis efectuado en el EDA**

In [6]:
df['name_coll'][:3]

,name_coll
0,Toy Story Collection
1,None
2,Grumpy Old Men Collection


In [4]:
# Extraer el primer actor de la lista en la columna 'actors'
df['first_actor'] = df['actors'].apply(lambda x: x.split(',')[0] if pd.notna(x) else '')

# Extraer el primer director de la lista en la columna 'director'
df['first_director'] = df['director'].apply(lambda x: x.split(',')[0] if pd.notna(x) else '')

In [5]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,name_gen,production_companies,production_countries,languages,return,release_year,actors,director,first_actor,first_director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,None,...,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English,12.451801,1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,Tom Hanks,John Lasseter
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"Adventure,Fantasy,Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français",4.043035,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,Robin Williams,Joe Johnston
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"Romance,Comedy","Warner Bros., Lancaster Gate",United States of America,English,0.000000,1995,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,Walter Matthau,Howard Deutch
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English,5.090760,1995,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,Whitney Houston,Forest Whitaker
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English,0.000000,1995,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,Steve Martin,Charles Shyer


In [ ]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'name_coll', 'name_gen',
       'production_companies', 'production_countries', 'languages', 'return',
       'release_year', 'actors', 'director', 'first_actor', 'first_director'],
      dtype='object')

Se utilizará el atributo overview en combinación de otros más para evaluar los distintos modelos

In [7]:
df = df.fillna('')  # Replace None with empty strings

##**Model1**

In [8]:
#Extrar las columnas relevantes para el modelo
model1 = df[['title', 'overview']].copy()

In [9]:
# Crear una instancia de TfidfVectorizer
tfidf_1 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

In [10]:
# Aplicar la transformación TF-IDF al contenido
tfidf_matriz_1 = tfidf_1.fit_transform(model1['overview'])


In [14]:
# Función para obtener recomendaciones
def recomendacion_m1(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model1.index, index=model1['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model1.duplicated(['title']).any():
            primer_ind = model1[model1['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidf_matriz_1[ind], tfidf_matriz_1).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model1)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model1.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

##**Se realizan distintas pruebas para evaluar el modelo**

In [84]:

recomendacion_m1('The Dark Knight Rises')

['Batman Forever',
 'The Dark Knight',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [85]:

recomendacion_m1('Jumanji')

['Word Wars', 'The Bar', 'Table No. 21', 'Quintet', 'The Dark Angel']

In [86]:

recomendacion_m1('Titanic')

['Grantham and Rose',
 'The Legend of 1900',
 'Raise the Titanic',
 'Dustbin Baby',
 'The Experience']

In [88]:

recomendacion_m1('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'Nightbeast',
 'The Machinist',
 'Escape from New York']

In [87]:

recomendacion_m1('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [89]:

recomendacion_m1('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'Prince of Persia: The Sands of Time',
 'Doctor in Distress']

In [90]:
# Se extraen las columnas necesarias para el Sistema de Recomendación
model1 = df[['title', 'overview']]
model1.to_parquet('movies_model1.parquet')

In [91]:
#Descargar el archivo
from google.colab import files
#files.download('movies_model1.parquet')

##**Model2**

In [94]:
#Extrar las columnas relevantes para el modelo
model2 = df[['title', 'overview', 'first_director']].copy()

In [23]:
# Crear una instancia de TfidfVectorizer
tfidf_2 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

In [24]:
model2['concatenado'] = model2['overview'] + ' ' + model2['first_director']

In [25]:
# Aplicar la transformación TF-IDF
tfidf_matriz_2 = tfidf_2.fit_transform(model2['concatenado'])

In [27]:
# Función para obtener recomendaciones
def recomendacion_m2(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model2.index, index=model2['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model2.duplicated(['title']).any():
            primer_ind = model2[model1['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidf_matriz_2[ind], tfidf_matriz_2).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model2)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model2.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones


In [28]:
recomendacion_m2('Titanic')

['Grantham and Rose',
 'Ghosts of the Abyss',
 'The Legend of 1900',
 'Avatar 2',
 'Dustbin Baby']

In [29]:
recomendacion_m2('Avatar')

['Aliens of the Deep',
 'Avatar 2',
 'Bloodbrothers',
 'Stand by Me Doraemon',
 'Ghosts of the Abyss']

In [30]:
recomendacion_m2('Wonder Woman')

['Wide Eyed and Legless',
 'Escape from New York',
 'The Machinist',
 'Green Lantern: First Flight',
 'Our Dancing Daughters']

In [31]:
recomendacion_m2('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [32]:
recomendacion_m2('The Avengers')

['The Right Kind of Wrong',
 'The Work and the Glory',
 'Benny & Joon',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure"]

In [33]:
recomendacion_m2('Shrek')

['Shrek 2',
 'Shrek the Third',
 'Silk Stockings',
 'Scared Shrekless',
 'Shrek Forever After']

##**Model3**

In [95]:
#Extrar las columnas relevantes para el modelo
model3 = df[['title', 'overview', 'name_gen', 'first_director']].copy()

In [36]:
##Se separan los géneros y se convierten en palabras individuales
model3['name_gen'] = model3['name_gen'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [37]:
# Se crea una instancia de la clase TfidfVectorizer
tfidf_3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidf_matriz_3 = tfidf_3.fit_transform(model3['overview'] + ' ' + model3['name_gen'] + ' ' + model3['first_director'])

In [38]:
tfidf_matriz_3.shape

(45345, 1108211)

In [39]:
# Función para obtener recomendaciones
def recomendacion_m3(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model3.index, index=model3['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model1.duplicated(['title']).any():
            primer_ind = model3[model3['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidf_matriz_3[ind], tfidf_matriz_3).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model3)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model3.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [40]:
recomendacion_m3('Titanic')

['Grantham and Rose',
 'Ghosts of the Abyss',
 'The Legend of 1900',
 'Dustbin Baby',
 'Raise the Titanic']

In [41]:
recomendacion_m3('Avatar')

['Avatar 2',
 'Aliens of the Deep',
 'Bloodbrothers',
 'Stand by Me Doraemon',
 'The War of the Robots']

In [42]:
recomendacion_m3('Wonder Woman')

['DC Showcase: Catwoman',
 'Green Lantern: First Flight',
 'Green Lantern: Emerald Knights',
 'Superman: Doomsday',
 'Wide Eyed and Legless']

In [43]:
recomendacion_m3('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace']

In [44]:
recomendacion_m2('The Avengers')

['The Right Kind of Wrong',
 'The Work and the Glory',
 'Benny & Joon',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure"]

##**Se decide probar con los atributos 'title', 'name_gen', 'first_actor', 'first_director' esperando tener una posible mejora en cuanto a la precisión de las recomendaciones**

##**Model4**

In [96]:
#Extrar las columnas relevantes para el modelo
model4 = df[['title', 'name_gen', 'first_actor', 'first_director']].copy()

In [46]:
#Se separan los géneros y se convierten en palabras individuales
model4['name_gen'] = model4['name_gen'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [47]:
#Se crea una instancia de la clase TfidfVectorizer
tfidf_4 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidf_matriz_4 = tfidf_4.fit_transform(model4['name_gen'] + ' ' + model4['first_actor'] + ' ' + model4['first_director'])

In [48]:
# Función para obtener recomendaciones
def recomendacion_m4(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model4.index, index=model4['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model4.duplicated(['title']).any():
            primer_ind = model4[model4['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidf_matriz_4[ind], tfidf_matriz_4).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model4)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model4.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [49]:
recomendacion_m4('Titanic')

['Avatar 2', 'Jude', 'Sense and Sensibility', 'A Little Chaos', 'The Reader']

In [50]:
recomendacion_m4('Avatar')

['Avatar 2',
 'Clash of the Titans',
 'Wrath of the Titans',
 'Drift',
 "The Hunter's Prayer"]

In [51]:
recomendacion_m4('Shrek')

['Shrek 2',
 'Shrek the Third',
 'Shrek the Halls',
 'The Chronicles of Narnia: Prince Caspian',
 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe']

In [52]:
recomendacion_m4('Star Wars')

['Return of the Jedi',
 'The Empire Strikes Back',
 'Comic Book: The Movie',
 'Time Runner',
 'Star Wars: Episode I - The Phantom Menace']

In [53]:
recomendacion_m4('Toy Story')

['Toy Story 2', 'Toy Story of Terror!', 'Luxo Jr.', 'Tin Toy', "Red's Dream"]

In [54]:
recomendacion_m4('Star Trek')

['Star Trek Into Darkness',
 'Star Trek Beyond',
 'Blind Dating',
 'Jack Ryan: Shadow Recruit',
 'People Like Us']

In [55]:
recomendacion_m4('Mars')

['Blazing a Trail to the Stars',
 'Fire on the Mountain',
 'A Paralyzing Fear: The Story of Polio in America',
 'The Dancemaker',
 'Home Page']

In [56]:
recomendacion_m4('Avatar')

['Avatar 2',
 'Clash of the Titans',
 'Wrath of the Titans',
 'Drift',
 "The Hunter's Prayer"]

In [57]:
recomendacion_m4('The Hunger Games')

['The Hunger Games: Mockingjay - Part 1',
 'The Hunger Games: Mockingjay - Part 2',
 'The Hunger Games: Catching Fire',
 'Passengers',
 'Joy']

In [58]:
recomendacion_m4('Forrest Gump')

['Cast Away',
 'The Polar Express',
 'Larry Crowne',
 'Sleepless in Seattle',
 'The Money Pit']

In [59]:
recomendacion_m4('Men in Black')

['Men in Black II',
 'The Sunset Limited',
 'The Good Old Boys',
 'The Three Burials of Melquiades Estrada',
 'Wild Wild West']

In [92]:
recomendacion_m4('Wonder Woman')

['Green Lantern: Emerald Knights',
 'DC Showcase: Catwoman',
 'Superman: Doomsday',
 'Green Lantern: First Flight',
 'Justice League: Crisis on Two Earths']

##**Model5**

In [97]:
#Extrar las columnas relevantes para el modelo
model5 = df[['title', 'name_gen', 'tagline' ,'first_actor', 'first_director']].copy()

In [61]:
#Se separan los géneros y se convierten en palabras individuales
model5['name_gen'] = model5['name_gen'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
#Se separan los slogans y se convierten en palabras individuales
model5['tagline'] = model5['tagline'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
#Se crea una instancia de la clase TfidfVectorizer
tfidf_5 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
#Aplicar la transformación TF-IDF y obtener matriz numérica
tfidf_matriz_5 = tfidf_5.fit_transform(model5['name_gen'] + ' ' + model5['tagline'] + ' ' + model5['first_actor']+ ' ' + model5['first_director'])

In [65]:
# Función para obtener recomendaciones
def recomendacion_m5(titulo):
  #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model5.index, index=model5['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model5.duplicated(['title']).any():
            primer_ind = model5[model5['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidf_matriz_5[ind], tfidf_matriz_5).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model5)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model5.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [66]:
recomendacion_m5('Toy Story')

['Toy Story 2', 'Luxo Jr.', 'Tin Toy', "Red's Dream", 'Knick Knack']

In [67]:
recomendacion_m5('Wonder Woman')

['DC Showcase: Catwoman',
 'Green Lantern: Emerald Knights',
 'Superman/Batman: Apocalypse',
 'Superman: Doomsday',
 'The Magic of Ordinary Days']

In [68]:
recomendacion_m5('Avatar')

['Avatar 2',
 'Clash of the Titans',
 'The Shack',
 "The Hunter's Prayer",
 'Tinker Bell']

In [69]:
recomendacion_m5('Batman Returns')

['The Merry Gentleman',
 'Hansel and Gretel',
 'The Last Time',
 'Dark Shadows',
 'Live from Baghdad']

In [70]:
recomendacion_m5('Star Wars')

['The Star Wars Holiday Special',
 'The Galaxy Invader',
 'Comic Book: The Movie',
 "On Her Majesty's Secret Service",
 'The Empire Strikes Back']

In [71]:
recomendacion_m5('The Avengers')

['The Invisible Woman',
 'Onegin',
 'Get Smart',
 'Tall Tale',
 'The End of the Affair']

In [72]:
recomendacion_m5('The Dark Knight Rises')

['The Dark Knight', 'Batman Begins', 'Following', 'Harsh Times', 'Metroland']

In [73]:
recomendacion_m5('Jumanji')

["Fathers' Day",
 'Hook',
 'The World According to Garp',
 'The Pagemaster',
 'House of D']

In [74]:
recomendacion_m5('Titanic')

['Avatar 2',
 'Little Children',
 'Deep Sea 3D',
 'Christmas Carol: The Movie',
 'Sense and Sensibility']

In [75]:
recomendacion_m5('Men in Black')

['Men in Black II',
 'The Good Old Boys',
 'The Sunset Limited',
 'The Three Burials of Melquiades Estrada',
 'Men in Black 3']

##**Modelo seleccionado**
El modelo Model3 fue el que presentó una mejor recomendación pero tiene un mayor costo computacional. Es por ello que elegimos el segundo mejor modelo, Model5, que podrá adaptarse a las limitaciones de memoria del sitio render.

In [81]:
# Se extraen las columnas necesarias para el Sistema de Recomendación
model5 = df[['title', 'name_gen', 'tagline' ,'first_actor', 'first_director']]

In [82]:
# Se exportan los datos a formato parquet
model5.to_parquet('movies_model5.parquet')

In [83]:
model5.shape

(45345, 5)